# Diplodatos Kaggle Competition

## Importamos librerías y funciones

In [1]:
# Import the required packages
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from libs_IAA import *

In [ ]:
def getDuplicateColumns(df): 
    duplicateColumnNames = set() 
    for x in range(df.shape[1]): 
        col = df.iloc[:, x] 
        for y in range(x + 1, df.shape[1]):  
            otherCol = df.iloc[:, y] 
            if col.equals(otherCol): 
                duplicateColumnNames.add(df.columns.values[y])  
    return list(duplicateColumnNames) 

## Dataset

In [3]:
df_train = pd.read_csv("./data/train.csv")
df_train['is_train_set'] = 1
df_test = pd.read_csv("./data/test.csv")
df_test['is_train_set'] = 0
df_train.TripType.replace({999: 0}, inplace=True)
df_train.TripType = df_train.TripType*0.01

In [53]:
df_train

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,is_train_set
0,0.00,5,Friday,6.811315e+10,-1,FINANCIAL SERVICES,1000.0,1
1,0.08,9,Friday,1.070081e+09,1,IMPULSE MERCHANDISE,115.0,1
2,0.08,9,Friday,3.107000e+03,1,PRODUCE,103.0,1
3,0.08,9,Friday,4.011000e+03,1,PRODUCE,5501.0,1
4,0.08,10,Friday,6.414410e+09,1,DSD GROCERY,2008.0,1
...,...,...,...,...,...,...,...,...
453406,0.22,191344,Sunday,7.315096e+10,1,BEAUTY,3405.0,1
453407,0.22,191344,Sunday,6.505300e+10,1,WIRELESS,1712.0,1
453408,0.22,191344,Sunday,7.918131e+09,1,BEAUTY,3405.0,1
453409,0.08,191347,Sunday,4.190008e+09,1,DAIRY,1512.0,1


### Agregación y Limpieza

In [4]:
y = df_train[['VisitNumber', 'Weekday', 'TripType']].groupby(['VisitNumber', 'Weekday']).agg(lambda x:x.value_counts().index[0])['TripType'].values

In [5]:
df = pd.concat([df_train.drop('TripType', axis=1), df_test])
df.reset_index(inplace=True, drop=True)

In [6]:
df[df.Upc.isnull()].groupby(['VisitNumber', 'Weekday', 'DepartmentDescription'], as_index=False).count().DepartmentDescription.unique()

array(['PHARMACY RX'], dtype=object)

Solo hay nulos en UPC (agrupados) en _PHARMACY RX_. Rellenaremos esos nulos con algun valor random extraído del historial de Upc para entradas de _PHARMACY RX_

In [7]:
# df.Upc            = df.Upc.fillna(-1)
# df.FinelineNumber = df.FinelineNumber.fillna(-1)

In [8]:
values = df[(df.DepartmentDescription=='PHARMACY RX') & (~df.Upc.isnull())].Upc.values
df.Upc = df.Upc.apply(lambda x: np.random.choice(values) if (np.isnan(x)) else x)

Ahora rellenamos FileNumber  y DepartmentDescription con un método parecido.
Asociamos a los _FinelineNumber_ nulos, el valor asociado al Filenumber existente correspondiente Upc de su entrada. (Igual para DD)

In [9]:
valores = []
for i in np.where(df.FinelineNumber.isnull())[0]:
    valor = np.random.choice(df[(df.Upc == df.Upc.iloc[i]) & ~df.FinelineNumber.isnull()].FinelineNumber.values)
    valores.append([i,valor])
valores = np.asarray(valores)
fine = list(df.FinelineNumber)
for i in range(len(valores)):
    fine[(valores[i,0]).astype(int)] = valores[i,1]
df.FinelineNumber = fine

In [10]:
valores2 = []
for i in np.where(df.DepartmentDescription.isnull())[0]:
    valor = np.random.choice(df[(df.Upc == df.Upc.iloc[i]) & ~df.DepartmentDescription.isnull()].DepartmentDescription.values)
    valores2.append([i,valor])
valores2 = np.asarray(valores2)
Depa = list(df.DepartmentDescription)
for i in range(len(valores2)):
    Depa[(valores2[i,0]).astype(int)] = valores2[i,1]
df.DepartmentDescription = Depa

In [11]:
df.Weekday.replace({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}, inplace=True)

Creamos columnas nuevas

In [12]:
RowxVis       = dict(df.groupby(['VisitNumber'])['Upc'].count())
df['RowxVis'] = df['VisitNumber'].apply(lambda x:RowxVis.get(x,0))
RowxWee       = dict(df.groupby(['Weekday'])['Upc'].count())
df['RowxWee'] = df['Weekday'].apply(lambda x:RowxWee.get(x,0))
UpcxVis       = dict(df.groupby(['VisitNumber'])['Upc'].nunique())
df['UpcxVis'] = df['VisitNumber'].apply(lambda x:UpcxVis.get(x,0))
UpcxWee       = dict(df.groupby(['Weekday'])['Upc'].nunique())
df['UpcxWee'] = df['Weekday'].apply(lambda x:UpcxWee.get(x,0))
df['ItemNum'] = df.apply(lambda x: str(x.Upc).zfill(11)[-6:-1], axis=1).astype(float).astype(int)
ItemxVis      = dict(df.groupby(['VisitNumber'])['ItemNum'].nunique())
df['ItxVis']  = df['VisitNumber'].apply(lambda x:ItemxVis.get(x,0))
ItemxWee      = dict(df.groupby(['Weekday'])['ItemNum'].nunique())
df['ItxWee']  = df['Weekday'].apply(lambda x:ItemxWee.get(x,0))
df['ManNum']  = df.apply(lambda x: str(x.Upc).zfill(11)[:6], axis=1).astype(float).astype(int)
ManxVis       = dict(df.groupby(['VisitNumber'])['ManNum'].nunique())
df['ManxVis'] = df['VisitNumber'].apply(lambda x:ManxVis.get(x,0))
ManxWee       = dict(df.groupby(['Weekday'])['ManNum'].nunique())
df['ManxWee'] = df['Weekday'].apply(lambda x:ManxWee.get(x,0))
ScanxVis      = dict(df.groupby(['VisitNumber'])['ScanCount'].sum())
df['ScaxVis'] = df['VisitNumber'].apply(lambda x:ScanxVis.get(x,0))
ScanxWee       = dict(df.groupby(['Weekday'])['ScanCount'].sum())
df['ScaxWee'] = df['Weekday'].apply(lambda x:ScanxWee.get(x,0))

In [15]:
df = BinarAll(df, ['DepartmentDescription'], merge=True)

In [16]:
display(df)

,VisitNumber,Weekday,Upc,ScanCount,FinelineNumber,is_train_set,RowxVis,RowxWee,UpcxVis,UpcxWee,ItemNum,ItxVis,ItxWee,ManNum,ManxVis,...,DepartmentDescription_PLAYERS AND ELECTRONICS,DepartmentDescription_PLUS AND MATERNITY,DepartmentDescription_PRE PACKED DELI,DepartmentDescription_PRODUCE,DepartmentDescription_SEAFOOD,DepartmentDescription_SEASONAL,DepartmentDescription_SERVICE DELI,DepartmentDescription_SHEER HOSIERY,DepartmentDescription_SHOES,DepartmentDescription_SLEEPWEAR/FOUNDATIONS,DepartmentDescription_SPORTING GOODS,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS,DepartmentDescription_nan
0,5,5,6.811315e+10,-1,1000.0,1,1,96247,1,38136,2929,1,9561,681131,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9,5,1.070081e+09,1,115.0,1,3,96247,3,38136,727,3,9561,107008,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,9,5,3.107000e+03,1,103.0,1,3,96247,3,38136,3107,3,9561,3,3,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,9,5,4.011000e+03,1,5501.0,1,3,96247,3,38136,4011,3,9561,4,3,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,10,5,6.414410e+09,1,2008.0,1,3,96247,3,38136,235,3,9561,641441,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647049,191346,7,3.120033e+09,1,4639.0,0,17,133975,17,44141,3013,17,9704,312003,16,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647050,191346,7,3.700091e+09,1,8947.0,0,17,133975,17,44141,1229,17,9704,370009,16,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647051,191346,7,3.239000e+10,1,1118.0,0,17,133975,17,44141,1778,17,9704,323900,16,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647052,191346,7,7.874205e+09,1,1752.0,0,17,133975,17,44141,5336,17,9704,787420,16,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
df.drop('DepartmentDescription_nan', axis=1, inplace=True)

In [20]:
df2  = df.groupby(['VisitNumber', 'Weekday'], as_index=False).sum()

In [22]:
df2.drop(['Upc', 'FinelineNumber', 'ItemNum', 'ManNum'], axis=1, inplace=True)

In [23]:
df3 = df[['VisitNumber', 'Weekday', 'Upc', 'FinelineNumber', 'ItemNum', 'ManNum']
        ].groupby(['VisitNumber', 'Weekday'], as_index=False).agg(lambda x: x.mode()[0])

In [24]:
df4 = df[['VisitNumber', 'Weekday', 'ScanCount', 'RowxVis', 'RowxWee',
          'UpcxVis', 'UpcxWee', 'ItxVis', 'ItxWee', 'ManxVis', 'ManxWee',
         'ScaxVis', 'ScaxWee']
        ].groupby(['VisitNumber', 'Weekday'], as_index=False).median()

In [25]:
df5 = df3.merge(df4, on=['VisitNumber', 'Weekday'])

In [29]:
df = df2.merge(df5, on=['VisitNumber', 'Weekday'])

In [30]:
display(df)

,VisitNumber,Weekday,ScanCount_x,is_train_set,RowxVis_x,RowxWee_x,UpcxVis_x,UpcxWee_x,ItxVis_x,ItxWee_x,ManxVis_x,ManxWee_x,ScaxVis_x,ScaxWee_x,DepartmentDescription_1-HR PHOTO,...,Upc,FinelineNumber,ItemNum,ManNum,ScanCount_y,RowxVis_y,RowxWee_y,UpcxVis_y,UpcxWee_y,ItxVis_y,ItxWee_y,ManxVis_y,ManxWee_y,ScaxVis_y,ScaxWee_y
0,5,5,-1,1,1,96247,1,38136,1,9561,1,6180,-1,106945,0,...,6.811315e+10,1000.0,2929,681131,-1.000000,1.0,96247.0,1.0,38136.0,1.0,9561.0,1.0,6180.0,-1.0,106945.0
1,7,5,2,0,4,192494,4,76272,4,19122,4,12360,4,213890,0,...,7.410811e+09,4504.0,1099,605388,1.000000,2.0,96247.0,2.0,38136.0,2.0,9561.0,2.0,6180.0,2.0,106945.0
2,8,5,28,0,529,2213681,483,877128,460,219903,391,142140,644,2459735,0,...,2.238400e+09,1017.0,0,200661,1.217391,23.0,96247.0,21.0,38136.0,20.0,9561.0,17.0,6180.0,28.0,106945.0
3,9,5,3,3,9,288741,9,114408,9,28683,9,18540,9,320835,0,...,3.107000e+03,103.0,727,3,1.000000,3.0,96247.0,3.0,38136.0,3.0,9561.0,3.0,6180.0,3.0,106945.0
4,10,5,3,3,9,288741,9,114408,9,28683,9,18540,9,320835,0,...,2.800054e+09,115.0,235,280005,1.000000,3.0,96247.0,3.0,38136.0,3.0,9561.0,3.0,6180.0,3.0,106945.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,191343,7,9,7,49,937825,49,308987,49,67928,42,45451,63,1037050,0,...,3.400000e+09,5225.0,246,885306,1.285714,7.0,133975.0,7.0,44141.0,7.0,9704.0,6.0,6493.0,9.0,148150.0
95670,191344,7,5,5,25,669875,25,220705,25,48520,20,32465,25,740750,0,...,7.417002e+09,3405.0,1034,791813,1.000000,5.0,133975.0,5.0,44141.0,5.0,9704.0,4.0,6493.0,5.0,148150.0
95671,191345,7,17,0,169,1741675,169,573833,169,126152,143,84409,221,1925950,0,...,1.380310e+09,2764.0,201,370008,1.307692,13.0,133975.0,13.0,44141.0,13.0,9704.0,11.0,6493.0,17.0,148150.0
95672,191346,7,17,0,289,2277575,289,750397,289,164968,272,110381,289,2518550,0,...,4.072000e+03,4639.0,33,787420,1.000000,17.0,133975.0,17.0,44141.0,17.0,9704.0,16.0,6493.0,17.0,148150.0


In [38]:
df['Weekend']  = ((df.Weekday==6) | (df.Weekday==7)).astype(int)
df['Devol']    = (df.ScanCount_y < 0).astype(int)
df['Compra']   = (df.ScanCount_y != 0).astype(int)
df['Dev_Mean'] = (df.ScanCount_x < 0).astype(int)
df['Com_Mean'] = (df.ScanCount_x != 0).astype(int)
df['FineCut']  = pd.qcut(df.FinelineNumber, q=20, labels=False)
df             = BinarAll(df, 'FineCut', merge=True)
df             = BinarAll(df, 'Weekday', merge=True)

In [40]:
display(df)

,VisitNumber,ScanCount_x,is_train_set,RowxVis_x,RowxWee_x,UpcxVis_x,UpcxWee_x,ItxVis_x,ItxWee_x,ManxVis_x,ManxWee_x,ScaxVis_x,ScaxWee_x,DepartmentDescription_1-HR PHOTO,DepartmentDescription_ACCESSORIES,...,FineCut_12,FineCut_13,FineCut_14,FineCut_15,FineCut_16,FineCut_17,FineCut_18,FineCut_19,Weekday_1,Weekday_2,Weekday_3,Weekday_4,Weekday_5,Weekday_6,Weekday_7
0,5,-1,1,1,96247,1,38136,1,9561,1,6180,-1,106945,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,7,2,0,4,192494,4,76272,4,19122,4,12360,4,213890,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
2,8,28,0,529,2213681,483,877128,460,219903,391,142140,644,2459735,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,9,3,3,9,288741,9,114408,9,28683,9,18540,9,320835,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,10,3,3,9,288741,9,114408,9,28683,9,18540,9,320835,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,191343,9,7,49,937825,49,308987,49,67928,42,45451,63,1037050,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
95670,191344,5,5,25,669875,25,220705,25,48520,20,32465,25,740750,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
95671,191345,17,0,169,1741675,169,573833,169,126152,143,84409,221,1925950,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
95672,191346,17,0,289,2277575,289,750397,289,164968,272,110381,289,2518550,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


In [42]:
duplicateColNames = getDuplicateColumns(df) 
print('Duplicate Columns are :') 
for column in duplicateColNames : 
    print('Column Name : ', column)
    
df.drop('Com_Mean', axis=1, inplace=True)

Duplicate Columns are :
Column Name :  Com_Mean
Column Name :  Dev_Mean


## Sets: Entrenamiento, Validación y Test.

In [69]:
X_train = df[df.is_train_set != 0].drop('is_train_set', axis=1)
X_test  = df[df.is_train_set == 0].drop('is_train_set', axis=1)

In [70]:
SS_tr = StandardScaler()
SS_tr.fit(X_train)
X_train_SS_tr = SS_tr.transform(X_train)
X_test_SS_tr  = SS_tr.transform(X_test)

In [71]:
random_state = 20

In [72]:
print('Ratio Test/Train:', len(X_test_SS_tr)/len(X_train_SS_tr))

Ratio Test/Train: 0.42735234003192646


In [73]:
x_train, x_valid, y_train, y_valid = train_test_split(
    X_train_SS_tr, y, test_size=0.18, random_state=random_state)
print('Tamaño X_train:', x_train.shape)
print('Tamaño X_valid:', x_valid.shape)
print('Tamaño X_test :', X_test.shape)

Tamaño X_train: (54963, 57)
Tamaño X_valid: (12066, 57)
Tamaño X_test : (28645, 57)


## Entrenamiento

In [74]:
xgb = XGBClassifier(random_state=random_state, learning_rate=0.02, n_estimators=600,
              objective='binary:logistic', subsample=0.8, min_child_weight=5,
              max_depth=5, gamma=1.5, colsample_bytree=0.8)

In [ ]:
# xgb = XGBClassifier(random_state=random_state, learning_rate=0.08, max_depth=None)
xgb.fit(x_train, y_train, eval_set=[(x_valid,y_valid)], early_stopping_rounds=25)

[0]	validation_0-merror:0.53879
Will train until validation_0-merror hasn't improved in 25 rounds.
[1]	validation_0-merror:0.52536
[2]	validation_0-merror:0.51724
[3]	validation_0-merror:0.51666
[4]	validation_0-merror:0.51392
[5]	validation_0-merror:0.51359
[6]	validation_0-merror:0.51036
[7]	validation_0-merror:0.51044
[8]	validation_0-merror:0.50671
[9]	validation_0-merror:0.50547
[10]	validation_0-merror:0.50497
[11]	validation_0-merror:0.50381
[12]	validation_0-merror:0.50307
[13]	validation_0-merror:0.50340
[14]	validation_0-merror:0.50456
[15]	validation_0-merror:0.50216
[16]	validation_0-merror:0.50124
[17]	validation_0-merror:0.50141
[18]	validation_0-merror:0.50232
[19]	validation_0-merror:0.50157
[20]	validation_0-merror:0.50149
[21]	validation_0-merror:0.50124
[22]	validation_0-merror:0.50091
[23]	validation_0-merror:0.49992
[24]	validation_0-merror:0.49983
[25]	validation_0-merror:0.49867
[26]	validation_0-merror:0.49884
[27]	validation_0-merror:0.49710
[28]	validation_0-m

## Validación

In [62]:
print(accuracy_score((y_train*100).astype(int), (xgb.predict(x_train)*100).astype(int)))
print(accuracy_score((y_valid*100).astype(int), (xgb.predict(x_valid)*100).astype(int)))

0.7791241380565107
0.7286590419360186


## Resultados

In [76]:
yNew = xgb.predict(np.asarray(X_test_SS_tr))
print((yNew*100).astype(int))

[ 9 40 21 ...  8 39 39]


In [77]:
pd.Series((yNew*100).astype(int)).value_counts()

8     4052
39    3580
9     2885
0     2187
40    2122
      ... 
4       81
23      38
12      31
41      21
43       4
Length: 36, dtype: int64

In [78]:
yy = (yNew*100).astype(int)

yy = pd.Series(yy)

yy.replace({0:999}, inplace=True)

## Submit

In [79]:
X_test  = df[df.is_train_set == 0].drop('is_train_set', axis=1)

In [80]:
submission = pd.DataFrame(list(zip(X_test.VisitNumber, yy)), columns=["VisitNumber", "TripType"])

In [81]:
submission.to_csv("../data/submission.csv", header=True, index=False)